In [6]:
import numpy as np
import pandas as pd
import pathlib
import matplotlib as mpl
import matplotlib.pyplot as plt
import sys
import os.path
from matplotlib.ticker import MaxNLocator
import warnings

In [7]:
cwd = pathlib.Path().resolve()
src = cwd.parent

# Replace with your own directory path
root = src.parent
data = src.parent.parent.parent
data_path = os.path.join(os.path.join(data, 'data_tana'), 'other_stations')

sys.path.append(str(src))
sys.path.append(str(root))
sys.path.append(str(data_path))

path = os.path.join(data_path, 'Tana_data.xlsx')
sheet = pd.ExcelFile(path)
sheetlist = sheet.sheet_names
print(sheetlist)

['Sheet1', 'Sheet3', 'Sheet4', 'Sheet5', 'Sheet6', 'Sheet7', 'Sheet8', 'Sheet9', 'Sheet10', 'Sheet11', 'Sheet12', 'Sheet15', 'Sheet17', 'Sheet18', 'Sheet25', 'Sheet26', 'Sheet29', 'Sheet31', 'Sheet32', 'Sheet36', 'Sheet37', 'Sheet41', 'Sheet43', 'Sheet46', 'Sheet49', 'Sheet50', 'Sheet52', 'Sheet68']


In [8]:
def get_station_name(df):
    column_index = 1
    column_names = df.columns.tolist()
    column_name = column_names[column_index]
    parts = column_name.split('-')
    extracted_name = parts[0].strip()  
    return extracted_name

In [9]:
names_stations = []
for name in sheetlist:    
    df = sheet.parse(name)
    names_stations.append(get_station_name(df))
    #print(get_name)

print(names_stations)
#print(get_station_name(df))

#df.info()

['4BD01 MATHIOYA', '4BC04 RWAMUTHAMBI', '4BE10 TANA RUKANGA', '4BB01 RAGATI', '4AC03 SAGANA', '4AA05 SAGANA', '4AB06 AMBONI', '4AC04 NEW CHANIA', '4AA04 NAIROBI', '4AA02 THEGO', '4AA01 SAGANA', '4AB01 MURINGATO', '4BC05 RWAMUTHAMBI', '4BE04 KAYAHWE', '4CB04 THIKA', '4BF01 SABA SABA', '4AD01 GURA', '4BD07 MATHIOYA', '4BE01 MARAGUA', '4CA03 CHANIA', '4CA19 KARIMINU', '4DA14 KAMWETI', '4DA13 KIRINGA', '4DB05 NYAMINDI', '4DA10 THIBA', '4BC05 RWAMUTHAMBI', '4DD02 THIBA', '4EA03 KITHINO RIVER']


In [12]:
for i in range(0, len(sheetlist)):
    #print(sheet_index)
    df = pd.read_excel(path, sheet_name=sheetlist[i])
    #print(sheetlist[0])
    columns_to_drop = [2, 3, 4]  
    df = df.drop(df.columns[columns_to_drop], axis=1)
    df.columns = [
        "waterlevel(m)" if i == 1 else
        "Time_Q" if i == 2 else
        "discharge(m3/s)" if i == 3 else col
        for i, col in enumerate(df.columns)
    ]
    df['Time'] = pd.to_datetime(df['Time'], format='mixed')
    df['Time_Q'] = pd.to_datetime(df['Time_Q'], format='mixed')

    time_check = True

    df_level = df.iloc[:,0:2].dropna()
    df_discharge = df.iloc[:,2:4].dropna()

    df_discharge = df_discharge.rename(columns={"Time_Q": "Time"})


    merged_dataframe = pd.merge_asof(df_level, df_discharge, on="Time", tolerance=pd.Timedelta("1days"))
    merged_dataframe.head()

    path = os.path.join(data_path, f"Tana_station_{names_stations[i]}.csv")
    merged_dataframe.to_csv(path, index = False)

ValueError: Excel file format cannot be determined, you must specify an engine manually.

In [ ]:
'''df = pd.read_excel(path, sheet_name=sheetlist[0])

# Drop specified columns and rename remaining columns
columns_to_drop = [2, 3, 4]  
df = df.drop(df.columns[columns_to_drop], axis=1)
df.columns = [
    "waterlevel(m)" if i == 1 else
    "Time_Q" if i == 2 else
    "discharge(m3/s)" if i == 3 else col
    for i, col in enumerate(df.columns)
]

# Convert 'Time' and 'Time_Q' columns to datetime format
df['Time'] = pd.to_datetime(df['Time'], format='mixed')
df['Time_Q'] = pd.to_datetime(df['Time_Q'], format='mixed')

# Check and adjust data based on condition
time_check = True
while time_check:
    condition_met = df['Time'][0] > df['Time_Q'][0]
    if condition_met:
        df.iloc[:, 1:3] = df.iloc[:, 1:3].shift(-1)
        df = df.iloc[:-1]
    else:
        time_check = False

# Split dataframe into level and discharge dataframes
df_level = df.iloc[:, 0:2].dropna()
df_discharge = df.iloc[:, 2:4].dropna()

# Initialize a new column in df_level for matched measurements
df_level['matched_measurement'] = np.nan

# Get timestamps from both dataframes
timestamps1 = df_level['Time'].values
timestamps2 = df_discharge['Time_Q'].values

# Broadcast and calculate the time differences
diffs = np.abs(timestamps1[:, None] - timestamps2[None, :])
min_indices = np.argmin(diffs, axis=1)

# Match measurements based on closest timestamps
matched_measurements = df_discharge.loc[min_indices, 'discharge(m3/s)'].values

# Add matched measurements to df_level occasionally
interval = 5  # Define the interval for adding measurements
for i in range(0, len(matched_measurements), interval):
    df_level.loc[i, 'matched_measurement'] = matched_measurements[i]

path = os.path.join(data_path, 'test_excel.csv')
df_level.to_csv(path, index = False)'''